## Agentic RAG
Retrieval agents are helpful when we need the system to decide whether it should look up information from a knowledge base or not.

To build a retrieval agent, we just need to allow the language model (LLM) to use a retriever tool that can search and return relevant information when needed.

## Setup

Let's get all required packages and imports

In [1]:
%%capture --no-stderr
%pip install -U --quiet langchain langchain-community langchain-groq langchain-core langgraph chromadb sentence-transformers langchain-huggingface huggingface_hub python-dotenv beautifulsoup4 tiktoken

In [2]:
import os
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

## RETRIEVER
Let's create a vector store retriever using the `Chroma` vector store. We will fetch documents from urls array and use the `Chroma` vector store to create a retriever.

In [3]:
urls = [
    "https://www.webmd.com/allergies/allergy-basics",
    "https://www.webmd.com/arthritis/understanding-arthritis-symptoms",
    "https://www.webmd.com/cancer/understanding-cancer-basics",
    "https://www.webmd.com/depression/what-is-depression",
    "https://www.webmd.com/cold-and-flu/flu-cold-symptoms",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
chunks = text_splitter.split_documents(docs_list)

model = "sentence-transformers/all-mpnet-base-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="medical_chroma_db"
)

retriever = vectorstore.as_retriever()

/tmp/ipykernel_2348/770961446.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model)
/workspaces/agentic-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template=(
        "You are a medical assistant. Use the context to analyze the user's symptoms.\n"
        "Context:\n{context}\n"
        "\n"
        "Question: {query}\n"
        "\n"
        "Return your answer in this format:\n"
        "Disease: <diagnosed disease>\n"
        "Criticality: <mild | moderate | severe>\n"
        "Remedy: <advice or home remedy or consult doctor>"
    )
)

llm = ChatGroq(
    model="llama-3.1-8b-instant",  
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

rag_retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.1})

# Ensure the format_docs function is defined
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Ensure the fallback_chain is defined

fallback_chain = RunnableLambda(lambda x: "Sorry, I couldn't find relevant information to answer your query based on the available medical knowledge.")

# Ensure the rag_chain_lcel is defined using the adjusted retriever
rag_chain_primary = (
    {"context": rag_retriever | format_docs, "query": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

rag_chain_lcel = rag_chain_primary.with_fallbacks([fallback_chain])

# Define a medical query that is expected to retrieve relevant documents
medical_query_lcel = "I have a fever and body ache."

# Invoke the rag_chain_lcel with the medical query and print the response
medical_response_lcel = rag_chain_lcel.invoke(medical_query_lcel)
print(medical_response_lcel)

Based on the symptoms you've described, I would analyze them as follows:

Disease: Flu
Criticality: Moderate
Remedy: Since you have a fever and body ache, it's likely that you have the flu. I would recommend consulting your doctor for proper diagnosis and treatment. In the meantime, you can try to manage your symptoms with rest, hydration, and over-the-counter medications such as pain relievers and decongestants. However, if your fever lasts more than three days or if you experience severe body aches, it's essential to consult your doctor to rule out any other underlying conditions.
